In [64]:
# import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
# %matplotlib inline

In [65]:
import datetime
import pickle as cPickle
import csv
import numpy as np
import random
import sys
from file_util import create_folder
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [66]:
# name_dataset        = 'news-19_paragraph_swap-random-1m'
# name_dataset        = 'headline_swap_news_v2'
# name_dataset        = 'headline_swap_news_v2.5'
# name_dataset        = 'headline_swap_news_v2.5_mf8'
name_dataset        = 'headline'
path_raw_data       = '../data/raw/' + name_dataset + '/'
path_processed_data = '../data/' + name_dataset + '/whole/'

In [67]:
create_folder(path_raw_data)
create_folder(path_processed_data + '/train')
create_folder(path_processed_data + '/dev')
create_folder(path_processed_data + '/test')
create_folder(path_processed_data + '/debug')
create_folder(path_processed_data + '/real_world')

In [5]:
print(path_raw_data)

../data/raw/headline/


In [6]:
print (path_processed_data)

../data/headline/whole/


# get term-/document-frequency

In [68]:
csv_reader = csv.reader(open(path_raw_data + 'train.csv', encoding="utf8"), delimiter='\t')

tkn2tf = {}
len_heads = [] #1
len_paras = [] #2
cnt_paras = [] #3
len_bodys = [] #4

# csv data: 0:id, 1:head, 2:body, 3:label

print (datetime.datetime.now().isoformat())

for n, row in enumerate(csv_reader):
    if (n+1) % 100000 == 0: print (n+1),
    
    head = row[0].lower().strip()
    
    for tkn in head.split():
        if tkn in tkn2tf: tkn2tf[tkn] += 1
        else: tkn2tf[tkn] = 1
    len_heads.append(len(head.split())) #1
    
    body = row[0].lower().strip()
    tkn_para = []
    for para in body.split('<eop>'):
        if para and para != ' ':
            _para = para + '<eop>'
            len_para = len(_para.split())
            len_paras.append(len_para) #2
            tkn_para.append(_para)
    cnt_paras.append(len(tkn_para)) #3
    
    body_split = []
    for tkn in body.split():
        if tkn in tkn2tf: tkn2tf[tkn] += 1
        else: tkn2tf[tkn] = 1
        body_split.append(tkn)
    len_bodys.append(len(body_split)) #4
            
print (n+1), 'Done'
print (datetime.datetime.now().time())
print ('voca size :', len(tkn2tf))

2021-02-28T13:23:35.656687
736
13:23:36.379752
voca size : 45311


In [69]:
n

735

In [70]:
sorted_token = sorted(tkn2tf.items(), key=lambda kv: kv[1], reverse=True)
tkn2idx = {}
for idx, (tkn, _) in tqdm(enumerate(sorted_token)):
    tkn2idx[tkn] = idx + 2
tkn2idx['<unk>'] = 1
tkn2idx[''] = 0
if len(tkn2idx) == len(tkn2tf)+2:
    print (len(tkn2idx), 'No problem')
print 

print ('Show top-10 tkn:')
for tkn, freq in sorted_token[:10]:
    print (tkn,':',freq)
print ('')

45311it [00:00, 567913.00it/s]


45313 No problem
Show top-10 tkn:
<eos> : 42450
। : 36968
, : 22574
না : 7024
করে : 6546
এই : 5026
! : 3764
? : 3668
থেকে : 3442
’ : 3216



In [14]:
#Creating Dic_mincut.txt
with open("Output.txt", "w", encoding="utf8") as txt_file:
    for tkn, freq in sorted_token:
        txt_file.write(tkn + "\n")

In [71]:
with open(path_processed_data + 'dic_mincutN.txt', 'w', encoding='utf8') as f:
    for key in tkn2idx.keys():
        f.write(key+'\n')

In [72]:
tkn2tf_mincut5 = {}
for tkn, tf in tkn2tf.items():
    if tf < 30:
        continue
    tkn2tf_mincut5[tkn] = tf
print ('voca size :', len(tkn2tf_mincut5))

voca size : 3308


In [73]:
tkn2tf_mincut5['<EOS>'] = tkn2tf_mincut5['<eos>']
tkn2tf_mincut5['<EOP>'] = tkn2tf_mincut5['<eop>']

del tkn2tf_mincut5['<eos>']
del tkn2tf_mincut5['<eop>']

In [74]:
import operator
sorted_voca = sorted(tkn2tf_mincut5.items(), key=operator.itemgetter(1))

In [75]:
list_voca_mincut = []
list_voca_mincut.append('')   # PAD
list_voca_mincut.append('<UNK>')   # UNK
list_voca_mincut.append('<EOS>')   # EOS
list_voca_mincut.append('<EOP>')   # EOP

for word, idx in sorted_voca:
    if word=='<UNK>' or word=='<EOP>' or word=='<EOS>':
        print("existing word", word)
        continue
    else:
        list_voca_mincut.append(word)
        
len(list_voca_mincut)

existing word <EOP>
existing word <EOS>


3310

In [76]:
with open(path_processed_data + 'dic_mincutN.txt', 'w', encoding='utf8') as f:
    for i in range(len(list_voca_mincut)):
        f.write(list_voca_mincut[i]+'\n')

In [77]:
dic_voca = {}
for voca in list_voca_mincut:
    dic_voca[voca] = len(dic_voca)

In [78]:
print(dic_voca[''], dic_voca['<UNK>'], dic_voca['<EOS>'], dic_voca['<EOP>'])

0 1 2 3


In [79]:
with open(path_processed_data + 'dic_mincutN.pkl', 'wb') as f:
    cPickle.dump(dic_voca, f)

## read voca from dic

In [80]:
list_voca = []
with open(path_processed_data + 'dic_mincutN.txt', 'r', encoding='utf8') as f:
    list_voca = f.readlines()
    list_voca = [x.strip() for x in list_voca]

dic_voca = {}
for voca in list_voca:
    dic_voca[voca] = len(dic_voca)

len(dic_voca)

3310

## for data processing

In [81]:
import copy
dic_voca_lower = copy.deepcopy(dic_voca)

In [82]:
dic_voca_lower['<eos>'] = dic_voca_lower['<EOS>']
dic_voca_lower['<eop>'] = dic_voca_lower['<EOP>']

del dic_voca_lower['<EOS>']
del dic_voca_lower['<EOP>']

In [83]:
len(dic_voca_lower)

3310

In [84]:
print(dic_voca_lower[''], dic_voca_lower['<UNK>'], dic_voca_lower['<eos>'], dic_voca_lower['<eop>'])

0 1 2 3


## stats

In [85]:
import csv
import sys
import numpy as np

data= []
with open(path_raw_data + 'train.csv', 'r', encoding='utf8') as f:
    data_csv = csv.reader(f, delimiter='\t')
    for row in data_csv:
        data.append(row)

In [86]:
def print_info(data):
    print("mean", np.average(data))
    print("std", np.std(data))
    print("max", np.max(data))
    print("95.xx coverage", np.average(data) +  2*np.std(data) )
    print("99.73 coverage", np.average(data) +  3*np.std(data) )
    print("99.95 coverage", np.average(data) +  3.5*np.std(data) )
    print("99.99 coverage", np.average(data) +  4*np.std(data) )

In [87]:
head = [x[0].strip() for x in data]
head_len = [len(x.split()) for x in head]
print('head_len')
print_info(head_len)

head_len
mean 523.0951086956521
std 354.01989501019807
max 3653
95.xx coverage 1231.1348987160482
99.73 coverage 1585.1547937262462
99.95 coverage 1762.1647412313455
99.99 coverage 1939.1746887364443


In [88]:
body = [x[0].strip() for x in data]

In [89]:
body_len = [len(x.split()) for x in body ]
print('body_len')
print_info(body_len)

body_len
mean 523.0951086956521
std 354.01989501019807
max 3653
95.xx coverage 1231.1348987160482
99.73 coverage 1585.1547937262462
99.95 coverage 1762.1647412313455
99.99 coverage 1939.1746887364443


In [90]:
context_len = [len(x.split('<EOP>')) for x in body]
print('context_len')
print_info(context_len)

context_len
mean 1.5
std 0.5
max 2
95.xx coverage 2.5
99.73 coverage 3.0
99.95 coverage 3.25
99.99 coverage 3.5


In [91]:
body_sentence = []
for sent in body:
    sent = sent.split('<EOP>')
    body_sentence.extend(sent)
body_len = [ len(x.split()) for x in body_sentence ]    
print('body_sentence')
print_info(body_len)

body_sentence
mean 348.3967391304348
std 253.67898133088772
max 3366
95.xx coverage 855.7547017922102
99.73 coverage 1109.433683123098
99.95 coverage 1236.2731737885417
99.99 coverage 1363.1126644539856


# encode to numpy

In [92]:
def fit_length(data, max_len_t, max_len_b):
    data_t, data_b = data
    
    list_zeros = np.zeros(max_len_b, 'int32').tolist()
    fl_data_t = []
    for datum in data_t:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_t:
            fl_data_t.append( datum[:max_len_t] )
        else:
            fl_data_t.append( datum + list_zeros[:(max_len_t-_len)] )
            
    fl_data_b = []
    for datum in data_b:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_b:
            fl_data_b.append( datum[:max_len_b] )
        else:
            fl_data_b.append( datum + list_zeros[:(max_len_b-_len)] )
    
    np_data_t = np.asarray(fl_data_t, dtype='int32')
    np_data_b = np.asarray(fl_data_b, dtype='int32')
    
    data = [np_data_t, np_data_b]
    return data

In [93]:
csv_reader = csv.reader(open(path_raw_data + 'train.csv', 'r', encoding='utf8'), delimiter='\t')

print (datetime.datetime.now().isoformat())
ids = []
heads = []
bodys = []
labels = []
for n, row in enumerate(csv_reader):
    
#     if n <  3000000:
#         continue

#     if n >=  3000000:
#         continue
        
                
    if (n+1) % 10000 == 0: print (n+1,)
    
    ids.append(row[0])
    labels.append((row[0]))
    
    head = []
    for tkn in row[0].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
            
    heads.append(head)
    
    body = []
    for tkn in row[0].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
            
    bodys.append(body)
    
print (n+1, 'Done')
print (datetime.datetime.now().isoformat()) # ~5 mins

2021-02-28T13:26:36.577271
736 Done
2021-02-28T13:26:37.154727


In [94]:
print (datetime.datetime.now().isoformat())
[np_heads, np_bodys] = fit_length([heads, bodys], 25, 2100)
print (datetime.datetime.now().isoformat()) # ~3 mins

2021-02-28T13:26:38.739488
2021-02-28T13:26:38.959934


In [95]:
print (datetime.datetime.now().isoformat())
t_trainpath = path_processed_data + '/train/train_title.npy'
np.save(t_trainpath, np_heads)
b_trainpath = path_processed_data + '/train/train_body.npy'
np.save(b_trainpath, np_bodys)
l_trainpath = path_processed_data + '/train/train_label.npy'
np.save(l_trainpath, labels)
print (datetime.datetime.now().isoformat())

2021-02-28T13:26:40.262418
2021-02-28T13:26:41.045323


# devset

In [96]:
csv_reader = csv.reader(open(path_raw_data + 'dev.csv', 'r', encoding='utf8'), delimiter='\t')

print (datetime.datetime.now().isoformat())
ids_dev = []
heads_dev = []
bodys_dev = []
labels_dev = []
for n, row in enumerate(csv_reader):
    if (n+1) % 10000 == 0: print (n+1,)
    
    ids_dev.append(row[0])
    labels_dev.append((row[0]))
    
    head = []
    for tkn in row[0].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
    heads_dev.append(head)
    
    body = []
    for tkn in row[0].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
    bodys_dev.append(body)
    
print (n+1, 'Done')
print (datetime.datetime.now().isoformat())

2021-02-28T13:26:42.676961
65 Done
2021-02-28T13:26:42.766721


In [97]:
print (datetime.datetime.now().isoformat())
[np_heads_dev, np_bodys_dev] = fit_length([heads_dev, bodys_dev], 25, 2100)
print (datetime.datetime.now().isoformat()) # ~3 mins

2021-02-28T13:26:44.534480
2021-02-28T13:26:44.556386


In [98]:
print (datetime.datetime.now().isoformat())
t_trainpath = path_processed_data + '/dev/dev_title.npy'
np.save(t_trainpath, np_heads_dev)
b_trainpath = path_processed_data + '/dev/dev_body.npy'
np.save(b_trainpath, np_bodys_dev)
l_trainpath = path_processed_data + '/dev/dev_label.npy'
np.save(l_trainpath, labels_dev)
print (datetime.datetime.now().isoformat())

2021-02-28T13:26:46.886249
2021-02-28T13:26:46.900211


# testset

In [99]:
csv_reader = csv.reader(open(path_raw_data + '/test.csv', 'r', encoding='utf8'), delimiter='\t')

print (datetime.datetime.now().isoformat())
ids_dev = []
heads_dev = []
bodys_dev = []
labels_dev = []
for n, row in enumerate(csv_reader):
    if (n+1) % 10000 == 0: print (n+1,)
    
    ids_dev.append(row[0])
    labels_dev.append(row[0])
    
    head = []
    for tkn in row[0].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
    heads_dev.append(head)
    
    body = []
    for tkn in row[0].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
    bodys_dev.append(body)
    
print (n+1, 'Done')
print (datetime.datetime.now().isoformat())

2021-02-28T13:26:49.157176
65 Done
2021-02-28T13:26:49.242945


In [100]:
print (datetime.datetime.now().isoformat())
[np_heads_dev, np_bodys_dev] = fit_length([heads_dev, bodys_dev], 25, 2100)
print (datetime.datetime.now().isoformat()) # ~3 mins

2021-02-28T13:26:53.110848
2021-02-28T13:26:53.137775


In [102]:
print (datetime.datetime.now().isoformat())
t_trainpath = path_processed_data + '/test/test_title.npy'
np.save(t_trainpath, np_heads_dev)
b_trainpath = path_processed_data + '/test/test_body.npy'
np.save(b_trainpath, np_bodys_dev)
l_trainpath = path_processed_data + '/test/test_label.npy'
np.save(l_trainpath, labels_dev)
print (datetime.datetime.now().isoformat())

2021-02-28T13:26:57.054703
2021-02-28T13:26:57.125513


# debugset

In [103]:
print (datetime.datetime.now().isoformat())
t_trainpath = path_processed_data + '/debug/debug_title.npy'
np.save(t_trainpath, np_heads_dev[:200])
b_trainpath = path_processed_data + '/debug/debug_body.npy'
np.save(b_trainpath, np_bodys_dev[:200])
l_trainpath = path_processed_data + '/debug/debug_label.npy'
np.save(l_trainpath, labels_dev[:200])
print (datetime.datetime.now().isoformat())

2021-02-28T13:26:59.560404
2021-02-28T13:26:59.593314


In [104]:
with open(path_processed_data + 'dic_mincutN.txt', encoding='utf8') as f:
    test_list_voca = f.readlines()
    test_list_voca = [x.strip() for x in test_list_voca]

In [105]:
from nlp_vocab import Vocab

In [106]:
tt = Vocab(test_list_voca)

In [107]:
print(tt.index2sent(np_heads_dev[15]))

<UNK> <UNK> <UNK> নির্বাচনী প্রচারণা চান বাংলাদেশের <UNK> শেখ <UNK> রহমানের ছোট ভাই শেখ আবু <UNK> ছেলে শেখ <UNK> <UNK> সন্তান শেখ <UNK> <UNK> <UNK>
